# ***ทำนายราคา Bitcoin โดยใช้ Machine Learning and Python*** 

In [ ]:
#Install numpy and pandas 

import numpy as np
import pandas as pd

In [ ]:
#เลือกไฟล์ DataSet สามารถดาวโหลดจาก Blockchain.com ได้
from google.colab import files
uploaded = files.upload()

Saving market-price.csv to market-price (3).csv


In [ ]:
#เก็บข้อมูลลงชุดตัวแปล
df = pd.read_csv('market-price (3).csv')
df.head(7)


,Timestamp,market-price
0,08/20/10 0:00,0.07
1,08/23/10 0:00,0.07
2,08/26/10 0:00,0.07
3,08/29/10 0:00,0.06
4,09/01/10 0:00,0.06
5,09/04/10 0:00,0.06
6,09/07/10 0:00,0.06


In [ ]:
#ลบคอลัมน์วันที่ออกจากกรอบข้อมูล
df.drop(['Timestamp'], 1, inplace=True)


In [ ]:
#แสดง 7 แถวแรกของชุดข้อมูลใหม่
df.head(7)


,market-price,Prediction
0,0.07,0.28
1,0.07,0.29
2,0.07,0.29
3,0.06,0.29
4,0.06,0.28
5,0.06,0.25
6,0.06,0.23


In [ ]:
#สร้างตัวแปรสําหรับการทํานาย 'n' วันออกในอนาคต ('n' เป็นจํานวนเต็มโดยพลการ) และคอลัมน์ที่เรียกว่า 'คาดการณ์' ที่จะมีราคาของ Bitcoin 30 วันจากราคาปัจจุบัน (ราคาของ BTC ในคอลัมน์ราคา)
prediction_days = 30 #n = 30 days
df['Prediction'] = df[['market-price']].shift(-prediction_days)


In [ ]:
#แสดง 7 แถวแรกของชุดข้อมูลใหม่
df.head(7)

,market-price,Prediction
0,0.07,0.28
1,0.07,0.29
2,0.07,0.29
3,0.06,0.29
4,0.06,0.28
5,0.06,0.25
6,0.06,0.23


In [ ]:
#สร้างชุดข้อมูลอิสระ นี่คือชุดที่มีคุณลักษณะในการทําการคาดการณ์ในอนาคตด้วย ก่อนอื่นเราจะแปลง dataframe 
#ไปเป็นอาร์เรย์ตัวเลขและวางคอลัมน์คาดการณ์แล้วเราจะลบแถว 'n' สุดท้ายที่จากชุดข้อมูล ในบทความนี้หมายความว่าเราจะลบ 30 วันล่าสุดตั้งแต่ 'n' = prediction_days ซึ่งเท่ากับ 30
X = np.array(df.drop(['Prediction'],1))
X= X[:len(df)-prediction_days]
print(X)


[[7.00000e-02]
 [7.00000e-02]
 [7.00000e-02]
 ...
 [9.38570e+03]
 [9.78574e+03]
 [9.16726e+03]]


In [ ]:
#สร้างชุดข้อมูลที่ขึ้นอยู่กับนี้จะเป็นชุดข้อมูลที่มีเป้าหมายของเรานั่นคือข้อมูลที่เรากําลังจะทํานาย เราจะบรรลุผลสําเร็จนี้โดยการแปลง dataframe 
#เป็นอาร์เรย์ตัวเลขและได้รับทั้งหมดของค่าจากคอลัมน์การทํานายของ dataframeจากนั้นเราจะได้ค่าทั้งหมดจากชุดข้อมูลที่สร้างขึ้นยกเว้นแถว 'n' ล่าสุด
y = np.array(df['Prediction'])
y = y[:-prediction_days] 
print(y)

[2.800000e-01 2.900000e-01 2.900000e-01 ... 1.177438e+04 1.229372e+04
 1.186582e+04]


In [ ]:
#แยกข้อมูลออกเป็น 80% การฝึกอบรมและ 20% ทดสอบชุดข้อมูล
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
 #สร้างตัวแปรที่เรียกว่าprediction_days_arrayและตั้งค่าเท่ากับแถว 30 สุดท้ายของชุดข้อมูลเดิมจากคอลัมน์ราคา
prediction_days_array = np.array(df.drop(['Prediction'],1))[-prediction_days:]
print(prediction_days_array)

[[ 8899.66]
 [ 9569.21]
 [ 9450.84]
 [ 9658.04]
 [ 9670.43]
 [ 9775.15]
 [ 9459.97]
 [ 9426.7 ]
 [ 9380.03]
 [ 9284.78]
 [ 9276.58]
 [ 9004.23]
 [ 9133.97]
 [ 9072.42]
 [ 9348.91]
 [ 9238.04]
 [ 9295.9 ]
 [ 9193.51]
 [ 9174.71]
 [ 9392.66]
 [ 9551.28]
 [11042.4 ]
 [11114.93]
 [11077.77]
 [11750.28]
 [11767.6 ]
 [11392.43]
 [11774.38]
 [12293.72]
 [11865.82]]


In [ ]:
#สร้างรูปแบบการถดถอยแบบเวกเตอร์สนับสนุนโดยใช้ฟังก์ชันพื้นฐานรัศมี (rbf) และฝึกแบบจําลอง
from sklearn.svm import SVR
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.00001)#Create the model
svr_rbf.fit(x_train, y_train) #Train the model

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=1e-05,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
#ทดสอบความถูกต้องของโมเดลบนชุดข้อมูลการทดสอบ
svr_rbf_confidence = svr_rbf.score(x_test, y_test)
print("svr_rbf accuracy: ", svr_rbf_confidence)

svr_rbf accuracy:  0.8568564606366953


In [ ]:
#พิมพ์โมเดลที่คาดการณ์ไว้สําหรับชุดข้อมูลการทดสอบและค่าที่แท้จริงของราคาของบิตคอยน์
svm_prediction = svr_rbf.predict(x_test)
print(svm_prediction)

print()

print(y_test)

[1.01803391e+04 6.15774459e+02 1.03339811e+01 2.56389712e+02
 2.64866250e+02 6.14839903e+02 6.26683895e+02 8.62319336e+03
 1.07939068e+01 3.32293765e+02 6.57593861e+00 6.83566874e+03
 9.29658179e+00 4.59025422e+03 8.16850663e+03 3.74416670e+02
 6.12541506e+02 6.33932011e+00 3.94035719e+02 6.10844893e+02
 6.36455525e+00 6.34230078e+00 1.13023607e+02 4.89674342e+02
 3.15226694e+02 6.43272182e+03 6.06807917e+02 6.93777910e+00
 5.54241199e+03 2.60942214e+02 2.63511457e+02 7.87821869e+03
 1.69188764e+02 8.69103688e+00 4.65610847e+03 7.23948279e+00
 2.78377892e+02 2.96684850e+02 6.98863138e+00 4.70476540e+02
 6.37646598e+00 8.10885200e+03 8.03705403e+03 9.42449210e+00
 4.57986609e+03 8.77786411e+03 4.03795743e+02 9.12629246e+03
 8.11881972e+03 2.54134163e+02 7.58682584e+03 1.14198870e+01
 1.82792923e+02 7.35027763e+00 1.31079922e+01 4.43250231e+02
 6.37024369e+00 6.34366047e+00 5.55915626e+03 2.56640889e+02
 8.29445700e+03 4.50886826e+02 3.06141638e+02 6.30244499e+02
 1.77875030e+03 2.714083

In [ ]:
#พิมพ์แบบจําลองของการคาดการณ์ราคา Bitcoin สําหรับถัดไป 30 วัน.
svm_prediction = svr_rbf.predict(prediction_days_array)
print(svm_prediction)


[9444.7032455  8335.5321031  8496.7563405  8288.3535776  8295.00232964
 8498.02313967 8485.7604614  8523.23643289 8565.04437715 8648.8626712
 8658.75457606 9344.93883469 8966.19289783 9155.99539572 8589.37359189
 8715.80400279 8636.51292251 8806.35446433 8852.55728828 8554.68788552
 8358.76157367 8038.59032364 8086.61408988 8053.613495   8648.93663826
 8629.14494343 8669.54577719 8621.35416468 7784.79380812 8511.17036292]


In [ ]:
#พิมพ์ราคาที่แท้จริงของบิตคอยน์สําหรับ 30 วันถัดไป
df.tail(prediction_days)


,market-price,Prediction
1189,8899.66,NaN
1190,9569.21,NaN
1191,9450.84,NaN
1192,9658.04,NaN
1193,9670.43,NaN
1194,9775.15,NaN
1195,9459.97,NaN
1196,9426.70,NaN
1197,9380.03,NaN
1198,9284.78,NaN


# ***เสร็จแล้วว🎉🎉✨✨***

---

